In [ ]:
import csv
import pywikibot
from pywikibot.data import api
import pprint

site = pywikibot.Site("wikidata", "wikidata")
repo = site.data_repository()

item_titles = []
with open('namen.csv', newline='') as csvfile:
    reader = csv.DictReader(csvfile, delimiter=';', quotechar='"')
    for row in reader:
        #print(row)
        #print(row['Vorname'], row['Nachname'])
        names = {}
        if row['Vorname'] != '' and row['Nachname'] != '':
            names['name'] = row['Vorname'] + " " + row['Nachname']
        else:
            names['name'] = None
        
        if row['Vorname'] != '' and row['Nachname'] != '' and row['Namenszusatz'] != '':
             names['name_with_affix'] = row['Vorname'] + " " + row['Namenszusatz'] + " " + row['Nachname']
        else:
             names['name_with_affix'] = None
       
        item_titles.append(names)
        
print(item_titles)

def get_items(site, item_title):
    """
    Requires a site and search term (item_title) and returns the results.
    """
    params = {"action": "wbsearchentities",
              "format": "json",
              "language": "en",
              "type": "item",
              "search": item_title}
    request = api.Request(site=site, **params)
    return request.submit()

def get_qids(search_results):
    qids = []
    [qids.append(result["id"]) for result in search_results["search"]]
    return qids
        
i = 0
for names in item_titles:
    if names['name'] != None or names['name_with_affix'] != None:
        qids = []
        if names['name'] != None:
            search_results = get_items(site, names['name'])
            qids += get_qids(search_results)
        if names['name_with_affix'] != None:
            search_results = get_items(site, names['name_with_affix'])
            qids += get_qids(search_results)
        qids = set(qids)

        if len(search_results["search"]) == 0:
            print("An item with the name %s don't exist." % names['name'])
            i += 1
        else:
            print("An item with the name %s may already exist." % names['name'])
    else:
        print("No qualified name available to search.")
        
print("%d judges are not on wikidata" % i)